In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/hannserr/schulich_data_science/main/nurse_shift_costs.csv')

In [5]:
# Create a new model
model = gp.Model('Nurse Scheduling Optimization')

# Create lists for cost values
Nurse_Title = df['Category'].tolist()
Weekday_Cost = df['Cost_Weekday'].tolist()
Weekend_Cost = df['Cost_Weekend'].tolist()
Overtime_Cost = df['Cost_Overtime'].tolist()

# Constants
NUM_NURSES = 26  # Number of nurses
NUM_CATEGORIES = 3  # Number of nurse categories (SRN, RN, NIT)
HOURS_PER_SHIFT = 12
MAX_HOURS = 60
MIN_HOURS = 36
NUM_SHIFTS_PER_WEEK = 14


# Decision variables
shift_assignment = model.addVars(NUM_NURSES, NUM_SHIFTS_PER_WEEK, vtype=GRB.BINARY, name="shift_assignment")
overtime_shifts = model.addVars(NUM_NURSES, vtype=GRB.INTEGER, name="overtime_shifts")
total_overtime_shifts = model.addVar(vtype=GRB.INTEGER, name="Total_Overtime_Shifts")

# Objective function
model.setObjective(
    gp.quicksum(Weekday_Cost[i] * shift_assignment[i, j] for i in range(NUM_NURSES) for j in range(NUM_SHIFTS_PER_WEEK) if j % 2 == 0) +
    gp.quicksum(Weekend_Cost[i] * shift_assignment[i, j] for i in range(NUM_NURSES) for j in range(NUM_SHIFTS_PER_WEEK) if j % 2 == 1) +
    gp.quicksum(Overtime_Cost[i] * overtime_shifts[i] for i in range(NUM_NURSES)), GRB.MINIMIZE)

# Constraints
# Each shift must be staffed with at least 6 nurses
for j in range(NUM_SHIFTS_PER_WEEK):
    model.addConstr(gp.quicksum(shift_assignment[i, j] for i in range(NUM_NURSES)) >= 6, f"MinNurses_Shift{j}")

# Each nurse can work between 36 and 60 hours per week
for i in range(NUM_NURSES):
    model.addConstr(HOURS_PER_SHIFT * gp.quicksum(shift_assignment[i, j] for j in range(NUM_SHIFTS_PER_WEEK)) >= MIN_HOURS, f"MinHours_Nurse{i}")
    model.addConstr(HOURS_PER_SHIFT * gp.quicksum(shift_assignment[i, j] for j in range(NUM_SHIFTS_PER_WEEK)) <= MAX_HOURS, f"MaxHours_Nurse{i}")

# Each shift must include at least one SRN
for j in range(NUM_SHIFTS_PER_WEEK):
    model.addConstr(gp.quicksum(shift_assignment[i, j] for i in range(NUM_NURSES) if Nurse_Title[i] == 'SRN') >= 1, f"MinSRN_Shift{j}")

# Nurses cannot be scheduled for back-to-back shifts
for i in range(NUM_NURSES):
    for j in range(NUM_SHIFTS_PER_WEEK - 1):
        model.addConstr(shift_assignment[i, j] + shift_assignment[i, j + 1] <= 1, f"NoBackToBack_Nurse{i}_Shift{j}")

# Overtime shifts calculation
for i in range(NUM_NURSES):
    model.addConstr(overtime_shifts[i] >= gp.quicksum(shift_assignment[i, j] for j in range(NUM_SHIFTS_PER_WEEK)) - MIN_HOURS / HOURS_PER_SHIFT, f"Overtime_Nurse{i}")
    model.addConstr(total_overtime_shifts == gp.quicksum(overtime_shifts[i] for i in range(NUM_NURSES)), "TotalOvertime_New") 

# Optimize the model
model.optimize()

# Print the solution
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in range(NUM_NURSES):
        for j in range(NUM_SHIFTS_PER_WEEK):
            if shift_assignment[i, j].X > 0.5:
                if overtime_shifts[i].X > 0:
                    print(f"Nurse {i} (Category: {Nurse_Title[i]}) works shift {j} (Overtime)")
                else:
                    print(f"Nurse {i} (Category: {Nurse_Title[i]}) works shift {j} (Regular)")
else:
    print("No optimal solution found.")

print(f"Total cost: {model.ObjVal}")
print(f"Total overtime shifts: {total_overtime_shifts.X}")

print(f"Decision Variables: {sum(v.x for v in model.getVars())}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 470 rows, 391 columns and 3000 nonzeros
Model fingerprint: 0x1871ac81
Variable types: 0 continuous, 391 integer (364 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 26 rows and 1 columns
Presolve time: 0.00s
Presolved: 444 rows, 390 columns, 2298 nonzeros
Variable types: 0 continuous, 390 integer (364 binary)

Root relaxation: objective 2.941800e+04, 229 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    29418.000000


Solution count 1: 29418 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.941800000000e+04, best bound 2.941800000000e+04, gap 0.0000%
Optimal solution found:
Nurse 0 (Category: SRN) works shift 7 (Regular)
Nurse 0 (Category: SRN) works shift 9 (Regular)
Nurse 0 (Category: SRN) works shift 13 (Regular)
Nurse 1 (Category: RN) works shift 6 (Regular)
Nurse 1 (Category: RN) works shift 8 (Regular)
Nurse 1 (Category: RN) works shift 12 (Regular)
Nurse 2 (Category: SRN) works shift 8 (Regular)
Nurse 2 (Category: SRN) works shift 10 (Regular)
Nurse 2 (Category: SRN) works shift 12 (Regular)
Nurse 3 (Category: RN) works shift 1 (Overtime)
Nurse 3 (Category: RN) works shift 3 (Overtime)
Nurse 3 (Category: RN) works shift 7 (Overtime)
Nurse 3 (Category: RN) works shift 11 (Overtime)
Nurse 3 (Category: RN) works shift 13 (Overtime)
Nurse 4 (Category: RN) works shift 1 (Regular)
Nurse 4 (Category: RN) works shift 3 (Regular)
Nurse 4 (Category: RN) works shift 13 (Regular)
Nurse 5 (C